<img src="https://github.com/GodStorm91/mobydock/blob/master/mobydock/static/docker-logo.png?raw=true" width="10%"> ➕
<img src="https://github.com/Microsoft/azuredatastudio/blob/master/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="6.5%"> = ❤

# This PowerShell Notebook will walk you through how to setup a SQL-on-Linux instance in a Docker container, on your local [Windows] machine.

## Preparation Steps
0A) Before any of this can work, you must have Docker Destop running. 
    You must also have the latest SqlServer module installed from the PowerShell Gallery.
Install-Module SqlServe

In [0]:
Import-Module SqlServer;

0C) Create a local directory for the container to use to store database files, outside of the container.

In [2]:
mkdir C:/SQLData/Docker/SQLDev02

mkdir : An item with the specified name C:\SQLData\Docker\SQLDev03 already exists.
At line:1 char:5
+ . { mkdir C:/SQLData/Docker/SQLDev03 }
+     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ResourceExists: (C:\SQLData\Docker\SQLDev03:String) [New-Item], IOException
    + FullyQualifiedErrorId : DirectoryExist,Microsoft.PowerShell.Commands.NewItemCommand
 


## Create your SQL-on-Linux Docker Container
1) Create a SQL-on-Linux Docker Container with just the code below.
    NOTE: You may want to change the password used for the SA account.
    You may also want to change the Volumne path: "C:\SQLData\Docker\SQLDev02"

In [3]:
docker run -d -p 10002:1433 -v C:\SQLData\Docker\SQLDev02:/sqlserver -e ACCEPT_EULA=Y -e SA_PASSWORD=Test1234 --name testcontainer02 mcr.microsoft.com/mssql/server:2019-latest

11c14b35246bd0fc3a78819ed4c64d3a53829d5a73c921d4eaed3f8b9772c58d


Verify that the SQL instance in the container is up & running

In [6]:
$SqlCred = (Get-Credential sa)
Get-SqlDatabase -ServerInstance 'localhost,10002' -Credential $SqlCred


Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
master               Normal        6.44 MB 1008.00 KB Simple       150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
model                Normal       16.00 MB    5.70 MB Full         150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
msdb                 Normal       13.88 MB    1.41 MB Simple       150 sa                        SQL_Latin1_General_CP1
                                       

2) Copy the backup file to the directory your container volume is mapped to.
    Make sure you use the location you stored the .bak file in.

In [9]:
Copy-Item -Path "$($Home)\Downloads\AdventureWorks2016.bak" -Destination C:\SQLData\Docker\SQLDev02

3) Restore the AdventureWorks2016 databas

In [12]:
Restore-SqlDatabase -ServerInstance 'localhost,10002' -Credential $SqlCred -BackupFile '/sqlserver/AdventureWorks2016.bak' -Database 'AdventureWorks2016' -AutoRelocateFile

In [14]:
Get-SqlDatabase -ServerInstance 'localhost,10002' -Credential $SqlCred


Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
AdventureWorks2016   Normal      209.63 MB    1.31 MB Simple       130 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
master               Normal        6.44 MB 1008.00 KB Simple       150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
model                Normal       16.00 MB    5.70 MB Full         150 sa                        SQL_Latin1_General_CP1
                                       

## Keep going, download & restore more databases

The code below allows you to grab all four AdventureWorksDW sample databases from GitHub, 
    then downloads and the .bak file.
    After the .bak files are downloaded you restore the instance specified.

In [0]:
$releases = Invoke-RestMethod https://api.github.com/repos/microsoft/sql-server-samples/releases
$BaksToDownload = ($releases | where {$_.name -eq 'AdventureWorks sample databases' -or $_.name -eq 'Wide World Importers sample database v1.0'}).assets |
WHERE { $_.name -like 'AdventureWorksDW201*bak' -and $_.name -notlike '*EXT*' } |
SELECT name, browser_download_url, size, updated_at
FOREACH( $BakInfo in $BaksToDownload )
{
    "$($BakInfo.name)";
    Invoke-WebRequest -Uri $BakInfo.browser_download_url -OutFile "C:\SQLData\Docker\SQLDev02\$($BakInfo.name)"
    Restore-SqlDatabase -ServerInstance 'localhost,10002' -Credential (Get-Credential sa) -BackupFile "/sqlserver/$($BakInfo.name)" -Database ($BakInfo.name -replace '.bak') -AutoRelocateFile
}$releases = Invoke-RestMethod https://api.github.com/repos/microsoft/sql-server-samples/releases
$BaksToDownload = ($releases | where {$_.name -eq 'AdventureWorks sample databases' -or $_.name -eq 'Wide World Importers sample database v1.0'}).assets |
WHERE { $_.name -like 'AdventureWorksDW201*bak' -and $_.name -notlike '*EXT*' } |
SELECT name, browser_download_url, size, updated_at

FOREACH( $BakInfo in $BaksToDownload )
{
    "$($BakInfo.name)";
    Invoke-WebRequest -Uri $BakInfo.browser_download_url -OutFile "C:\SQLData\Docker\SQLDev02\$($BakInfo.name)"
    Restore-SqlDatabase -ServerInstance 'localhost,10002' -Credential (Get-Credential sa) -BackupFile "/sqlserver/$($BakInfo.name)" -Database ($BakInfo.name -replace '.bak') -AutoRelocateFile
}